## 0. Set up

### Dependency

In [ ]:
!pip install -q -U bitsandbytes trl
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/DATA410'

/content/drive/MyDrive/Colab Notebooks/DATA410


### Login Huggingface

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Accelerator
[Accelerator: Fully Sharded Data Parallel](https://huggingface.co/docs/accelerate/usage_guides/fsdp)

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

## 1. Datasets

In [ ]:
!wget https://raw.githubusercontent.com/stop1one/StudyLLaMA/main/datasets/train.jsonl
!wget https://raw.githubusercontent.com/stop1one/StudyLLaMA/main/datasets/eval.jsonl

--2023-12-06 21:17:42--  https://raw.githubusercontent.com/stop1one/StudyLLaMA/main/datasets/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519135 (507K) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>] 506.97K  --.-KB/s    in 0.02s   

2023-12-06 21:17:42 (24.1 MB/s) - ‘train.jsonl’ saved [519135/519135]

--2023-12-06 21:17:42--  https://raw.githubusercontent.com/stop1one/StudyLLaMA/main/datasets/eval.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241379 (236K) [text/plain]

In [ ]:
from datasets import load_dataset
import json

train_dataset = load_dataset('json', data_files='train.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='eval.jsonl', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.
<</SYS>>

Question: {{Question}} [/INST]
Answer: {{Answer}}"""

def make_prompt(prompt, data, train=True):
    result = prompt.replace("{{Question}}", data["input"])
    if train:
        result = result.replace("{{Answer}}", data["output"])
    else:
        result = result.replace("{{Answer}}", "")
    return result

In [ ]:
prompted_train_text = {'text': []}
for d in train_dataset:
    prompted_train_text['text'].append(make_prompt(prompt, d, train=True))
prompted_eval_text = {'text': []}
for d in eval_dataset:
    prompted_eval_text['text'].append(make_prompt(prompt, d, train=True))

In [ ]:
from datasets import Dataset
prompted_train_dataset = Dataset.from_dict(prompted_train_text)
prompted_eval_dataset = Dataset.from_dict(prompted_eval_text)

## 2. Load LLaMA2-chat


In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments

base_model = "meta-llama/Llama-2-7b-chat-hf"

# compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # 뭐가 맞는지 모르겠음 그냥... 존나 억까다
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0} # Load the entire model on the GPU 0
)

model.config.use_cache = False #
model.config.pretraining_tp = 1 # 1이 아니면 더 정확하지만 느려짐

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

## 3. Tokenization

In [ ]:
from transformers import AutoTokenizer

# Tokenization
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    padding_side = "right", # Fix weird overflow issue with fp16 training
)
tokenizer.pad_token = tokenizer.eos_token

## 3.5. Zero-shot Test

In [ ]:
import random

example = random.choice(train_dataset)

example_prompt = make_prompt(prompt, example, train=False)
print(example_prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.
<</SYS>>

Question: How would you deal with irregular intervals or missing data in a time series? [/INST]
Answer: 


In [ ]:
model_input = tokenizer(example_prompt, return_tensors="pt")

model.eval()
with torch.no_grad():
    output = model.generate(**model_input, max_new_tokens=512, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1635: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.
<</SYS>>

Question: How would you deal with irregular intervals or missing data in a time series? [/INST]
Answer:  Thank you for asking! Dealing with irregular intervals or missing data in a time series is a common challenge in data analysis. There are several approaches I would use to address this issue.

Firstly, I would visually inspect the time series to identify any obvious patterns or trends that could help explain the irregular intervals or missing data. This could involve plotting the time series on a graph or using a statistical tool to vis

## 4. QLoRA

In [ ]:
def print_trainable_params(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

# PEFT parameters (LoraConfig)
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_params)
print_trainable_params(model)

model = accelerator.prepare_model(model)

trainable params: 33554432 || all params: 3533967360 || trainable%: 0.9494833591219133


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)

## 5. Training

In [ ]:
from datetime import datetime
from pytz import timezone

project_name = "Llama2-7b-chat-inst"
run_name = f"{project_name}-{datetime.now(timezone('Asia/Seoul')).strftime('%m-%d-%H-%M')}"
output_dir = "./" + run_name
print(run_name, output_dir)

Llama2-7b-chat-inst-12-07-14-27 ./Llama2-7b-chat-inst-12-07-14-27


In [ ]:
num_epochs = 15
batch_size = 2
gradient_accumulation_steps = 16
total_steps = len(prompted_train_dataset) // (batch_size * gradient_accumulation_steps) * num_epochs
total_steps

465

In [ ]:
import transformers

# Set training parameters
training_params = transformers.TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    max_steps=total_steps,
    learning_rate=1e-5,
    fp16=False,
    bf16=True, # V100: False, A100: True
    optim="paged_adamw_8bit",
    gradient_accumulation_steps=gradient_accumulation_steps,
    weight_decay=0.1,
    lr_scheduler_type='linear',
    warmup_steps=0,
    logging_dir="./logs",
    save_strategy="epoch",
    save_total_limit=3,
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
)

In [ ]:
from trl import SFTTrainer

# Mdoel fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=prompted_train_dataset,
    eval_dataset=prompted_eval_dataset,
    peft_config = peft_params,
    dataset_text_field = "text",
    max_seq_length = 512,
    tokenizer = tokenizer,
    args = training_params,
    packing = False,
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer,
        mlm=False
    ),
)

model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/481 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,No log,3.117763
1,No log,2.716098
2,No log,2.377427
4,No log,1.917094
5,No log,1.704954
6,No log,1.469537
8,No log,1.210750
9,No log,1.165733
10,No log,1.148379
12,No log,1.132068


Epoch,Training Loss,Validation Loss
0,No log,3.117763
1,No log,2.716098
2,No log,2.377427
4,No log,1.917094
5,No log,1.704954
6,No log,1.469537
8,No log,1.210750
9,No log,1.165733
10,No log,1.148379
12,No log,1.132068


TrainOutput(global_step=465, training_loss=1.7184349798387097, metrics={'train_runtime': 2526.5936, 'train_samples_per_second': 5.889, 'train_steps_per_second': 0.184, 'total_flos': 1.4495871795339264e+17, 'train_loss': 1.7184349798387097, 'epoch': 14.88})

### Save the model

In [ ]:
trainer.model.save_pretrained(run_name)
trainer.tokenizer.save_pretrained(run_name)

('Llama2-7b-chat-inst-12-07-14-27/tokenizer_config.json',
 'Llama2-7b-chat-inst-12-07-14-27/special_tokens_map.json',
 'Llama2-7b-chat-inst-12-07-14-27/tokenizer.json')

## 6. Inference Test

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from trl import SFTTrainer

base_model = "meta-llama/Llama-2-7b-chat-hf"

# compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0} # Load the entire model on the GPU 0
)

model.config.use_cache = False #
model.config.pretraining_tp = 1 # 1이 아니면 더 정확하지만 느려짐

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

# Tokenization
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    padding_side = "right", # Fix weird overflow issue with fp16 training
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "Llama2-7b-chat-inst-12-07-14-27")

In [ ]:
from transformers import pipeline

def inference_model(model, prompt, full=False):
    model.eval()
    with torch.no_grad():
        pipe = pipeline(
            task = "text-generation",
            model = ft_model,
            tokenizer = tokenizer,
            max_length = 1024,
            return_full_text=full,
        )
    model_output = pipe(prompt)
    return model_output

In [ ]:
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.
<</SYS>>

Question: {{Question}} [/INST]
Answer: {{Answer}}"""

def make_prompt(prompt, data, train=True):
    result = prompt.replace("{{Question}}", data["input"])
    if train:
        result = result.replace("{{Answer}}", data["output"])
    else:
        result = result.replace("{{Answer}}", "")
    return result

In [ ]:
import random

example = random.choice(train_dataset)

example_prompt = make_prompt(prompt, example, train=False)
print(example_prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.
<</SYS>>

Question: What is RAID structure in OS? What are the different levels of RAID configuration? [/INST]
Answer: 


In [ ]:
example_prompt = make_prompt(prompt, eval_dataset[96], train=False)
print(example_prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.
<</SYS>>

Question: What is Naive Bayes algorithm, When we can use this algorithm in NLP? [/INST]
Answer: 


In [ ]:
inf_test = inference_model(ft_model, example_prompt)

In [ ]:
print(inf_test[0]['generated_text'])

 Univariate analysis is the study of a single variable to analyze the data. Bivariate analysis is the study of two variables to analyze the data. In univariate analysis, we can identify the trends of a single variable. Bivariate analysis helps to identify the relationship between two variables. Bivariate analysis includes Pearson correlation coefficient, T-test, and regression analysis.


In [ ]:
print(make_prompt(prompt, example, train=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.
<</SYS>>

Question: Differentiate between univariate and bivariate analysis. [/INST]
Answer: The univariate analysis only summarizes one variable at a time. Measures of variance and central tendency are examples of univariate analysis. Bivariate analysis analyzes the relationship between two variablesâ€Š—â€Šsuch as correlation or covariance.


### Export the json file

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install jsonlines

In [ ]:
eval_dataset[0]

{'input': 'What is the purpose of Hash tables in statistics?',
 'output': 'When key-value pairs are stored in a hash table, the information regarding keys and associated values are stored in a hierarchical fashion using hash tables. The hashing function is used to provide an index that contains all of the information regarding keys and their associated values.'}

In [ ]:
prompted_test_text = {'input': [], 'text': []}
for d in eval_dataset:
    prompted_test_text['input'].append(d['input'])
    prompted_test_text['text'].append(make_prompt(prompt, d, train=False))

from datasets import Dataset
prompted_test_dataset = Dataset.from_dict(prompted_test_text)

In [ ]:
prompted_test_dataset[0]

{'input': 'What is the purpose of Hash tables in statistics?',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n[INST] <<SYS>>\nYou are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.\nPlease write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.\nPlease answer only the given one question without any additional information.\n<</SYS>>\n\nQuestion: What is the purpose of Hash tables in statistics? [/INST]\nAnswer: '}

In [ ]:
from tqdm import tqdm

inf_data = []
for data in tqdm(prompted_test_dataset):
    inf_output = inference_model(ft_model, data['text'], full=False)
    inf_data.append({'input': data['input'], 'output': inf_output[0]['generated_text']})

100%|██████████| 481/481 [4:05:25<00:00, 30.61s/it]


In [ ]:
inf_data

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
import os
path = "/content" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

In [ ]:
!pwd

/content


In [ ]:
import json

with open("inferences2.jsonl", encoding="utf-8", mode="w") as writer:
    for d in inf_data:
        writer.write(json.dumps(d) + "\n")

## 7. Graph-of-thoughts

In [ ]:
def make_prompt(prompt, data, keywords=False, explain=False, train=True):
    result = prompt
    if keywords:
        result = result.replace("{{Keywords}}", data["keywords"])
    else:
        result = result.replace("{{Question}}", data["input"])
        if train:
            result = result.replace("{{Answer}}", data["output"])
        else:
            result = result.replace("{{Answer}}", "")
    if explain:
        result = result.replace("{{Keywords}}", data["keywords"])
        result = result.replace("{{Explanations}}", data["explain"])

    return result

In [ ]:
prompt1 = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
[INST] <<SYS>>
You have expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the keywords of the following interview question to use your knowledge.
<</SYS>>

Example:
Question: What are the applications of Deep Learning?
Keywords of question:
Deep Learning, Computer vision, Natural language processing (NLP), Reinforcement learning

Referencing the example above, find the key keywords in the below question.
Question: {{Question}} [/INST]
Keywords of question:
"""

In [ ]:
prompt2 = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
[INST] <<SYS>>
You have expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please provide a simple but clear explanation for each of the keywords below to demonstrate depth of knowledge.
<</SYS>>

Keywords: {{Keywords}} [/INST]
Explanation of keywords:
"""

In [ ]:
prompt3 = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.

Question: {{Question}}
Keywords of question: {{Keywords}}
Explanations of keywords:
{{Explanations}}
<</SYS>>

Only refer to keywords or explanations when necessary; focus on providing an accurate answer to the given question.
 [/INST]
Answer: {{Answer}}"""

In [ ]:
import random

example = random.choice(train_dataset)

example_prompt = make_prompt(prompt1, example, train=False)
print(example_prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
[INST] <<SYS>>
You have expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the keywords of the following interview question to use your knowledge.
<</SYS>>

Example:
Question: What are the applications of Deep Learning?
Keywords of question:
Deep Learning, Computer vision, Natural language processing (NLP), Reinforcement learning

Referencing the example above, find the key keywords in the below question.
Question: Explain superclass in Java. [/INST]
Keywords of question:



In [ ]:
example = eval_dataset[96]

In [ ]:
example_prompt = make_prompt(prompt1, example, train=False)
print(example_prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
[INST] <<SYS>>
You have expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the keywords of the following interview question to use your knowledge.
<</SYS>>

Example:
Question: What are the applications of Deep Learning?
Keywords of question:
Deep Learning, Computer vision, Natural language processing (NLP), Reinforcement learning

Referencing the example above, find the key keywords in the below question.
Question: What is Naive Bayes algorithm, When we can use this algorithm in NLP? [/INST]
Keywords of question:



In [ ]:
inf_test = inference_model(ft_model, example_prompt)
print(inf_test[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

Naive Bayes algorithm, NLP, Text classification, Spam detection, Sentiment analysis


In [ ]:
keywords = inf_test[0]['generated_text']
example['keywords'] = keywords

In [ ]:
example_prompt2 = make_prompt(prompt2, example, keywords=True)
print(example_prompt2)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
[INST] <<SYS>>
You have expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please provide a simple but clear explanation for each of the keywords below to demonstrate depth of knowledge.
<</SYS>>

Keywords: Naive Bayes algorithm, NLP, Text classification, Spam detection, Sentiment analysis [/INST]
Explanation of keywords:



In [ ]:
inf_test2 = inference_model(ft_model, example_prompt2)
print(inf_test2[0]['generated_text'])

In [ ]:
explnation = inf_test2[0]['generated_text']
example['explain'] = explnation

In [ ]:
example_prompt3 = make_prompt(prompt3, example, explain=True, train=False)
print(example_prompt3)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

[INST] <<SYS>>
You are a job applicant with expertise in Mathematics, Statistics, Computer Science, and Machine Learning.
Please write the answer of the following interview question with a clear and comprehensive explanation to demonstrate depth of knowledge.
Please answer only the given one question without any additional information.

Question: What is Naive Bayes algorithm, When we can use this algorithm in NLP?
Keywords of question: Naive Bayes algorithm, NLP, Text classification, Spam detection, Sentiment analysis
Explanations of keywords:

1. Naive Bayes algorithm: Naive Bayes is a family of probabilistic supervised learning algorithms that are used for classification tasks. It is called "naive" because it assumes independence between features, even when they are not. Naive Bayes can be used for both binary and multiclass classification tasks.

Explanation of keywords:

2. N

In [ ]:
inf_test3 = inference_model(ft_model, example_prompt3)
print(inf_test3[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

### Random extraction

In [ ]:
import random

random_indices = random.sample(range(1, len(eval_dataset)), 50)
print(random_indices)

[308, 100, 234, 119, 428, 11, 261, 256, 187, 265, 408, 411, 67, 456, 159, 352, 162, 271, 326, 179, 422, 144, 203, 296, 401, 123, 437, 145, 230, 325, 415, 439, 130, 41, 96, 35, 332, 138, 365, 134, 383, 284, 448, 133, 216, 94, 447, 258, 307, 32]


In [ ]:
random_eval_dataset = [eval_dataset[i] for i in random_indices]
print(len(random_eval_dataset))

50


In [ ]:
random_eval_dataset[0]

{'input': 'Is it always necessary to use an 80:20 ratio for the train test split?',
 'output': 'No there is no such necessary condition that the data must be split into 80:20 ratio. The main purpose of the splitting is to have some data which the model has not seen previously so, that we can evaluate the performance of the model. If the dataset contains let’s say 50,000 rows of data then only 1000 or maybe 2000 rows of data is enough to evaluate the model’s performance.'}

In [ ]:
eval_dataset[0]

{'input': 'What is the purpose of Hash tables in statistics?',
 'output': 'When key-value pairs are stored in a hash table, the information regarding keys and associated values are stored in a hierarchical fashion using hash tables. The hashing function is used to provide an index that contains all of the information regarding keys and their associated values.'}

In [ ]:
from tqdm import tqdm


with open("llama2_got_results.jsonl", encoding="utf-8", mode="a") as writer:
    inf_data = []
    for data in tqdm(eval_dataset):
        # Make keywords
        step1 = make_prompt(prompt1, example, train=False)
        inf1 = inference_model(ft_model, step1, full=False)
        keywords = inf1[0]['generated_text']
        data['keywords'] = keywords
        step2 = make_prompt(prompt2, data, keywords=True)
        # Make explanations
        inf2 = inference_model(ft_model, step2, full=False)
        explanation = inf2[0]['generated_text']
        data['explain'] = explanation
        # Make answer
        step3 = make_prompt(prompt3, data, explain=True, train=False)
        inf3 = inference_model(ft_model, step3, full=False)
        answer = inf3[0]['generated_text']
        # Save data
        inf_data.append({'input': data['input'], 'output': answer, 'GT': data['output']})
        writer.write(json.dumps(inf_data[-1]) + "\n")


  0%|          | 0/481 [00:00<?, ?it/s]The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausa